In [1]:
import pathlib
import sys 
import random
import json
import cv2 
import base64
import requests as req
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from io import BytesIO  
%matplotlib inline
import sklearn.metrics.pairwise as pairwise
import time
import copy

#Importing ActionClassificationCosine

In [2]:
def weightedDistanceMatching(poseVector1,poseVector2):
    '''
    
    Parameters
    ----------
    (poseVector1 : a 2D list pose vector of a human + theta [[poseVec],[thetaVector]]
    theta : weigths for human pose vector
    Example theta = [w_0x , w_0y.........w_17x, w_17y])
    or 
    (poseVector1 : a 1D list pose vector of a human, used for building tree. 
    theta : A list with of ones. 
    Example [1,1,.....1] (17x1) )
    
    poseVector2 : pose vector that is to be compared with the human
    
    
    
    Returns
    ------------
    weigtheDistance  : 
    '''
    poseVector1 = np.array(poseVector1) 
    if poseVector1.shape[0] == 36:
        poseVector1 = poseVector1.reshape(1,-1)
        theta = np.ones_like(poseVector1)
    elif poseVector1.shape[0] == 72:
        poseVector1 = poseVector1.reshape(2,36)
        theta = poseVector1[1]
        poseVector1 = poseVector1[0]
    
    poseVector2 = np.array(poseVector2).reshape(1,-1)
    term1 = 1/ np.sum(theta)
    #Finding term 2
    distanceTranspose = np.absolute(poseVector1 - poseVector2).transpose()
    term2 = np.matmul(theta,distanceTranspose)

        
    
    return term1 * term2
    

In [3]:
def cosineDistanceMatching(poseVector1,poseVector2):
    '''
    Returns the cosine similarity as a distance function between the two L2 normalized vectors.
    The distance is inversely porportional to the similarity between the two vectors
    '''
    poseVector1 , poseVector2 = np.array(poseVector1).reshape(1,-1) , np.array(poseVector2).reshape(1,-1)
    cosineDistance = pairwise.cosine_distances(poseVector1 , poseVector2) 
    distance = np.sqrt(cosineDistance * 2)
    return distance

In [4]:
import import_ipynb
import ActionClassificationCosine 

importing Jupyter notebook from ActionClassificationCosine.ipynb
importing Jupyter notebook from ActionClassificationCosineDatasetGen.ipynb
Loading Poses Data....
Initializing VPTREE....
VPTREE Ready To Use....
importing Jupyter notebook from SinglePlayerPoseDatasetGen.ipynb


#Setting constants

In [5]:
#constants
PART_NAMES = [
    "nose", "leftEye", "rightEye", "leftEar", "rightEar", "leftShoulder",
    "rightShoulder", "leftElbow", "rightElbow", "leftWrist", "rightWrist",
    "leftHip", "rightHip", "leftKnee", "rightKnee", "leftAnkle", "rightAnkle"
]
PART_IDS = {pn: pid for pid, pn in enumerate(PART_NAMES)} ;PART_IDS
CONNECTED_PART_NAMES = [
    ("leftHip", "leftShoulder"), ("leftElbow", "leftShoulder"),
    ("leftElbow", "leftWrist"), ("leftHip", "leftKnee"),
    ("leftKnee", "leftAnkle"), ("rightHip", "rightShoulder"),
    ("rightElbow", "rightShoulder"), ("rightElbow", "rightWrist"),
    ("rightHip", "rightKnee"), ("rightKnee", "rightAnkle"),
    ("leftShoulder", "rightShoulder"), ("leftHip", "rightHip")
]

CONNECTED_PART_INDICES = [(PART_IDS[a], PART_IDS[b]) for a, b in CONNECTED_PART_NAMES];


#Helper functions

In [6]:
def get_adjacent_keypoints(keypoints):
    '''
    Helper function of draw_skel_and _kp
    Returns 2 coord of 2 points where line needs to be drawn
    EXAMPLE: [[X1,Y1],[X2,Y2]]
    '''
    results = []
    for left, right in CONNECTED_PART_INDICES:
       results.append(
           np.array([
                [ keypoints[left]['position']['x'] , keypoints[left]['position']['y'] ],
                [ keypoints[right]['position']['x'] , keypoints[right]['position']['y'] ]
                    ]
           ).astype(np.int32)
        )
    return results
            

In [7]:
def convertToBase64(image):
    retval, buffer = cv2.imencode('.jpg', image)
    my_string = base64.b64encode(buffer)
    my_string = my_string.decode('utf-8')
    return my_string

#Sending data to server
def getPoses(image):
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
    image_string = convertToBase64(image)
    url2 = 'http://localhost:3000/postImage'
    data = {'imgBase64':'data:image/png;base64,'+image_string}
    r = req.post(url=url2 , data = data)
    poses = r.json()
    
    url2 = 'http://localhost:3000/getBBox'
    r = req.post(url=url2)
    bboxes = r.json()
    return poses,bboxes

In [8]:
def draw_keypoints(poses,img):
    cv_keypoints= []
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. *score))
    out_img = cv2.drawKeypoints(img, cv_keypoints , outImage=np.array([]) ) 

In [9]:
def draw_skelton(poses,img):
    '''
    This function is not used anywhere. 
    Written to demonstrate how skelton can be drawn
    '''
    adjacent_keypoints = []
    out_img = img
    for pose in poses['detectionList']:
        keypoints  = pose['keypoints']
        new_keypoints = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoints)
    out_img = cv2.polylines(out_img , adjacent_keypoints, isClosed=False, color=(255,255,0))
    return out_img    

In [10]:
def draw_skel_and_kp(poses , img, color,classes):
    out_img = img
    adjacent_keypoints = []
    cv_keypoints = []
    #For every pose of the player
    for i,pose in enumerate(poses['detectionList']):
        keypoints = pose['keypoints']
        new_keypoint = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoint)
        
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. * score))
        cv2.putText(out_img, classes[i][0], (x,y), cv2.FONT_ITALIC, 0.6, (0, 255, 0), 1)
    
    out_img = cv2.drawKeypoints(
        img, cv_keypoints , outImage=np.array([]), color=color,
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS )
    out_img = cv2.polylines(out_img , adjacent_keypoints , isClosed=False, color=list(color))
    return out_img

#Sending to server

In [115]:
#Loading model
r = req.get(url='http://localhost:3000/loadModel')

In [82]:
700*1.77

1239.0

In [118]:
videoPath ='/Users/sandeep/Desktop/dataandmodles/data/unzoomed.mov'
cap = cv2.VideoCapture(videoPath)
count = 0
while cap.isOpened():
    ret,frame = cap.read()
    if ret :
        #do stuff
        start = time.time()
        poses,bboxes = getPoses(frame)
        end = time.time()
        print(f'Posenet Time:{round(end-start,3)}')
        
        if len(bboxes['bbox']) != 0:
            posesCopy = copy.deepcopy(poses)
        
            #frame = draw_visibilty_cone(poses,frame)
            start = time.time()
            classes = ActionClassificationCosine.fit(poses,
                                                     bboxes,
                                                     inputImgH=frame.shape[0],
                                                     inputImgW=frame.shape[1])
            end = time.time()
            print(f'VPN Time:{round(end-start,3)}')

            start = time.time()

            frame = draw_skel_and_kp(posesCopy,frame,[255,0,0],classes)
            end = time.time()
            print(f'Drawing Time:{round(end-start,3)}')
#         End of do stuff
        cv2.imshow('test', frame)
        #cv2.imwrite(f"/Users/sandeep/Desktop/dataandmodles/data/coneVision/{count}.png" , frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break  
            
    else:
        cap.release()
        break
    count += 1
    
cap.release()
cv2.destroyAllWindows()


Posenet Time:0.405
Posenet Time:0.422
Posenet Time:0.419
Posenet Time:0.391
Posenet Time:0.349
VPN Time:0.55
Drawing Time:0.001
Posenet Time:0.369
VPN Time:0.401
Drawing Time:0.001
Posenet Time:0.354
VPN Time:0.477
Drawing Time:0.002
Posenet Time:0.357
VPN Time:0.515
Drawing Time:0.002
Posenet Time:0.359
VPN Time:0.365
Drawing Time:0.001
Posenet Time:0.361
VPN Time:0.474
Drawing Time:0.002
Posenet Time:0.364
VPN Time:0.605
Drawing Time:0.001
Posenet Time:0.341
VPN Time:0.532
Drawing Time:0.002
Posenet Time:0.362
VPN Time:0.237
Drawing Time:0.001
Posenet Time:0.365
VPN Time:0.26
Drawing Time:0.001
Posenet Time:0.362
VPN Time:0.245
Drawing Time:0.001
Posenet Time:0.357
VPN Time:0.214
Drawing Time:0.001


In [43]:
bboxes

{'bbox': [[{'x': 703.3642535886137, 'y': 224.86874854275925},
   {'x': 1040.8136556552768, 'y': 224.86874854275925},
   {'x': 1040.8136556552768, 'y': 961.0684908467592},
   {'x': 703.3642535886137, 'y': 961.0684908467592}]]}

In [14]:
b = a.copy()


NameError: name 'a' is not defined

In [ ]:
a['a'] = 2

In [ ]:
a

In [ ]:
b